Мария Сухарева

## Loads

In [ ]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
!unrar x RNC_texts.rar
!rm RNC_texts.rar
!pip install corus

In [ ]:
!pip install tensorflow Cython matplotlib
!pip install git+https://github.com/lopuhin/python-adagram.git

In [ ]:
!pip install pymorphy2

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip
!unzip 196.zip -d 196
!pip install simple-elmo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
import numpy as np

from corus import load_morphoru_rnc

import adagram
from simple_elmo import ElmoModel

from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [7]:
morph = MorphAnalyzer()
word_token = RegexpTokenizer('\w+')

In [8]:
def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return word_token.tokenize(text)

## Corpus

Я использовала словарь МАС. Некотрые более менее похожие значения в словаре написаны через знак ||. Я вынесла их как отдельные значения в lexemes, но я ожидала, что модели объединят их в один класс, но как оказалось, так происходит далеко не всегда. В некоторых толкованиях нет примеров и само слово не употреблялось, а для того, чтобы применить модели, оно должно быть там, поэтому я добавила слово через дефис вначало к таким толкованиям.

In [9]:
lexemes = {
    'среда': ['Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами. Воздушная среда. Распространение света в однородной среде. Преломляющие среды. Фильтрующие среды.', 'Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма. Географическая среда.', 'Социально-бытовая обстановка, в которой протекает жизнь человека; окружение. [Нехлюдов] всем существом почувствовал отвращение к той своей среде, в которой он жил до сих пор. Л. Толстой, Воскресение. В судьбе, вкусах и характере каждого человека огромное значение имеет его детство, среда, в которой он вырастал, был воспитан. Соколов-Микитов, Автобиографические заметки.', 'кого или какая. Совокупность людей, связанных общностью социально-бытовых условий существования, общностью профессии, занятий. Рабочая среда. В среде учащихся. Читательская среда. □ — Из среды студенчества рекрутируется большинство революционеров. М. Горький, Жизнь Клима Самгина.', 'Среда - Третий (после воскресенья) день недели (середина недели).'],
    'мотив': ['Побудительная причина, основание, повод к какому-л. действию, поступку. Кроме повара, гневались и на лакеев, прислуживавших за столом. Мотивы были самые разнообразные; не так ступил, не так подал, не так взглянул. Салтыков-Щедрин, Пошехонская старина. — Был у меня и другой мотив желать сближения с вами: сходство наших убеждений. Чернышевский, Пролог.', 'Довод в пользу чего-л. Вольф стал очень оживленно выставлять мотивы, по которым дело должно было быть кассировано. Л. Толстой, Воскресение. Сабуров еще раз повторил все мотивы, по которым он решил отложить атаку до ночи. Симонов, Дни и ночи.', 'Составной элемент темы произведения искусства. Основной мотив повести. Вышивка по народным мотивам. □ Когда-то Пастухов купил несколько его картин — излюбленные гривнинские мотивы: вода, пепельные ветлы, дороги, уходящие в нежные дали. Федин, Костер.', 'Мотив - Муз. Простейшая ритмическая единица мелодии, состоящая обычно из двух-трех звуков, объединенных одним логическим акцентом.', 'Напев, мелодия. [Райский] сел за фортепьяно и начал брать сильные аккорды ---. Перебрал мотивы из нескольких опер. И. Гончаров, Обрыв. [Музыканты] подхватывали все тот же мотив мазурки. Л. Толстой, После бала. [Зина] попыталась припомнить их [Журбиных] песню — и слова и мотив упорно от нее ускользали. Кочетов, Журбины.'],
    'звезда': ['Небесное тело, состоящее из раскаленных газов (плазмы), по своей природе сходное с Солнцем и представляющееся взору человека на ночном небе светящейся точкой. Полярная звезда. Вечерняя звезда. □ Воздух был свежий и холодноватый, на чистом небе сияли крупные звезды. Достоевский, Братья Карамазовы.', 'перен. Судьба, участь; счастье, удача. Весть о солдатчине мало тревожила его: он верил в свою звезду. Помяловский, Очерки бурсы. — Это у вас счастливая звезда. Ведь на полсантиметра правее, и была бы совсем другая картина. А вы через месяц воевать сможете. Эренбург, Буря.', 'О человеке, прославившемся в какой-л. сфере деятельности; о знаменитости. — Ни одной зимы не проходило без того, чтобы не приезжала какая-нибудь звезда. Чехов, Живая хронология. На него [Урманова] еще в гимназии смотрели как на будущую звезду. Короленко, С двух сторон.', 'Геометрическая фигура с остроконечными выступами, равномерно расположенными по окружности; фигура с лучами, исходящими от центра. Пятиконечная звезда. Нарисовать звезду.', 'Предмет в форме подобной фигуры. [Потолок в кабаре] был черный, с большими звездами из серебряной бумаги. А. Н. Толстой, Хмурое утро. [Максим] водружает на стол маленькую рождественскую елку --- с блестящей звездой на маковке. Бек, События одной ночи.', 'Звезда - Воинский значок, имеющий такую форму и носимый на фуражке, шапке и т. п.', 'Знак отличия, орден, имеющий такую форму. Маршальская звезда. □ Их нагнал строгой наружности седой господин, со звездой на фраке. Писемский, Тысяча душ. [Багратион был] в новом узком мундире с русскими и иностранными орденами и с георгиевскою звездой на левой стороне груди. Л. Толстой, Война и мир. Он резко повернулся, и при этом на лацкане его пиджака блеснула Звезда Героя Социалистического Труда. С. Антонов, Три тысячи девятнадцатая морская.', 'Светлое пятно на лбу животного. Конь был рослый и статный, с белой звездой на лбу. Седых, Даурия.'],
    'земля': ['Третья от Солнца, обитаемая нами планета (с прописной буквы). Земля движется вокруг Солнца. Окружность Земли. Луна — спутник Земли. □ От Земли до Солнца около 150 миллионов километров; пролететь это расстояние — то же, что 4000 раз объехать кругом Земли. С. Вавилов, Глаз и Солнце.', 'Место жизни и деятельности людей. Слухом земля полнится. Поговорка. [Сальери:] Все говорят: нет правды на земле, Но правды нет — и выше. Пушкин, Моцарт и Сальери. И сколько вообще обидного на земле, чего вовсе не нужно. М. Горький, В людях.', 'Суша, земная твердь (в отличие от водного или воздушного пространства). В воображении невольно возникает мысль о близости берега, земли. Гл. Успенский, Очерки переходного времени. Ночь будет страшная, и буря будет злая, Сольются в мрак и гул и небо и земля. Фет, Приметы. По времени должен был уже показаться Сахалин, но никакой земли поблизости не было видно. Чаковский, У нас уже утро.', 'Верхний, поверхностный слой коры нашей планеты, а также ее более глубокие слои; почва, грунт. Пахать землю. В недрах земли. □ Мертвые в землю зарыты; больные Скрыты в землянках; рабочий народ Тесной гурьбой у конторы собрался. Н. Некрасов, Железная дорога. [Виктору] нетерпеливо хотелось в шахту, или, как он теперь говорил, под землю. — Под землей будем работать! — хвастался он Андрею. Горбатов, Донбасс.', 'Поверхность, плоскость, на которой мы стоим, по которой движемся. Веселый, свежий ветер гулял над землею и в меру шумел и играл, все шевеля и ничего не тревожа. Тургенев, Первая любовь. По тебе ль, нива, ветер разгуливал, Гнул колосья твои до земли. А. К. Толстой, Уж ты нива моя, нивушка.', 'Рыхлое темно-бурое вещество, входящее в состав коры нашей планеты. Песок с землей. Комок земли. □ Бешено кидаясь на берега, оно [море] оставляет рыб, ракушки и уносит песок, землю и прочее. И. Гончаров, Фрегат «Паллада». Тут он заметил, что окоп рядом засыпан свежей землей. Симонов, Пехотинцы.', 'Территория, находящаяся в чьем-л. владении, пользовании; обрабатываемая, используемая в сельскохозяйственных целях почва. Залежные земли. Пахотная земля. Колхозная земля. Помещичья земля. Церковные земли. Национализация земли. Освоение новых земель. □ — Земли мало [у крестьян]! я и так удивляюсь, как они концы с концами сводят. Тургенев, Бурмистр.', 'Высок. Страна, государство. [Пимен:] Да ведают потомки православных Земли родной минувшую судьбу. Пушкин, Борис Годунов. Все славянские земли волновались и готовились к восстанию. Тургенев, Накануне. После Батыева нашествия отшумела слава Киева. Русская земля стала расти и крепнуть вокруг Москвы. А. Н. Толстой, Стыд хуже смерти.', 'Земля - Устар. Поле, фон (ткани, обоев), по которому сделан рисунок.', 'Земля - Устарелое название буквы „з“.'],
    'светлый': ['Излучающий сильный свет (об источнике света). Светлая лампочка. □ Светлое, чудное солнце пристально смотрело на землю. Гл. Успенский, День нужды и скуки. Дитя не поворачивало головы за светлым лучом, проникавшим в комнату. Короленко, Слепой музыкант.', 'Хорошо освещенный, наполненный светом. Мои вещи были внесены в большую светлую комнату, выходившую большими окнами прямо на пруд. Мамин-Сибиряк, Сестры. Стояла светлая, лунная ночь, тротуары казались занесенными снегом, таким пронзительно белым был лунный свет. Павленко, Труженики мира.', 'Блестящий, излучающий блеск. Люблю тебя, булатный мой кинжал, Товарищ светлый и холодный. Лермонтов, Кинжал.', 'Не темного цвета. Светлое пальто. Светлые глаза. Светлые обои. □ Вошел полицейский чиновник красивой наружности, с бакенбардами не слишком светлыми и не темными. Гоголь, Нос. Здешняя осина имеет столь светлую кору, что я сначала принимал ее за березу. Арсеньев, В горах Сихотэ-Алиня.', 'Менее насыщенный, менее густой по сравнению с основным цветом. Светлая зелень. □ Сестра была блондинка, светлая блондинка. Достоевский, Подросток.', 'Чистый, прозрачный (о жидкости). Днепра ж светлы стремнины, Чиста его вода, Не видано в нем тины От веку никогда! А. К. Толстой, Сватовство. Лиза проворно поднялась и отерла глаза, на которых сияли светлые, непролившиеся слезы. Тургенев, Дворянское гнездо.', 'Чистый, ясный, не затуманенный (о глазах). Особенно хороши были светлые, черные, продолговатые глаза [Павла Петровича]. Тургенев, Отцы и дети.', 'Высокий, ясного чистого тембра (о голосе). [Голос Смирнова] — высокий баритон светлого тембра. Старк, Сценическая история опер П. И. Чайковского в б. Мариинском театре. — Борис! — крикнул светлый и холодный голос. — Ты бы шел в комнату, — дождь! М. Горький, Жизнь Матвея Кожемякина.', 'перен. Радостный, ничем не омраченный. Светлые воспоминания. Светлое будущее. □ За каждый светлый день иль сладкое мгновенье Слезами и тоской заплатишь ты судьбе. Лермонтов, Отчего. Быстро идет к нам новая, светлая жизнь. Чернышевский, Что делать? Настоящей, светлой, легкой веселости как будто никогда не было в его сердце. Достоевский, Подросток.', 'Выражающий радость, безмятежность. Светлая улыбка. □ — У вас такое веселое, светлое лицо, вы улыбаетесь. Тургенев, Дворянское гнездо.', 'перен. Чистый и ясный духом, умиротворенный, просветленный (о человеке). Светел, ясен проснулся на заре Степан Михайлович, весело крикнул свою Аришу. С. Аксаков, Семейная хроника. Склонилась мать, от радости светла: Впервые сын ее белоголовый Прошел один от стула до стола. Щипачев, Первые шаги.', 'Выражающий ясность духа, умиротворение. Лица торжественные, светлые, добрые; товарищи внимательны друг к другу и ласковы. Помяловский, Очерки бурсы.', 'перен. Обладающий высокими моральными качествами; возвышенный, благородный. Удивительно светел был этот человек, --- искренне веровавший в безграничную силу народа, — силу, способную творить чудеса. М. Горький, Н. Е. Каронин-Петропавловский.', 'Прекрасный, высокий. Перед рядами многих поколений Прошел твой светлый образ; чистых впечатлений И добрых знаний много сеял ты. Н. Некрасов, Сцены из лирической комедии «Медвежья охота».', 'перен. Ясный и логичный (об уме, мыслях). Светлый ум. □ — Книги ты читать горазд и писать горазд, мысли светлые, — знаю сама. А. Н. Толстой, Петр Первый. О Ефиме Лаврентьеве еще до войны упрочилась добрая слава человека, у которого «золотые руки» и светлая голова. Поповкин, Семья Рубанюк.', 'У христиан: относящийся к пасхе, пасхальный. Светлый праздник. Светлая заутреня. □ — Однажды слыхал я в ночь на светлое воскресенье, как печаловский колокол звонит. Куприн, Лесная глушь.', 'Устар. и народно-поэт. Употребляется как постоянный эпитет царя, боярина. [Купава:] Батюшка, светлый царь! [Царь:] Сказывай, слушаю! А. Островский, Снегурочка.']
    }

In [10]:
lexemes_norm = {}
for lexeme, meanings in lexemes.items():
    lexemes_norm[lexeme] = {'meanings': [], 'norm_meanings': []}
    lexemes_norm[lexeme]['meanings'] = meanings
    for meaning in meanings:
        lexemes_norm[lexeme]['norm_meanings'].append(normalize(meaning))

Я взяла Russian National Corpus, потому что в нем сразу есть морфологическая разметка, и мне не пришлось тратить время на нормализацию данных.

In [11]:
path = 'RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)

In [12]:
texts_with_lexemes = {}
for record in records:
    text = []
    norm_text = []
    for token in record.tokens:
        if token.text is not None:
            text.append(token.text)
        if token.pos != 'PUNCT':
            norm_text.append(token.lemma)
    for lexeme in lexemes.keys():
        if lexeme in norm_text:
            if lexeme not in texts_with_lexemes.keys():
                texts_with_lexemes[lexeme] = {'texts': [], 'norm_texts': []}
            else:
                texts_with_lexemes[lexeme]['texts'].append(' '.join(text))
                texts_with_lexemes[lexeme]['norm_texts'].append(norm_text)

## AdaGram

In [13]:
vm = adagram.VectorModel.load('/content/drive/MyDrive/4 курс/NLP 4 year/hw1/all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [ ]:
texts_meanings = {}
for lexeme, texts in texts_with_lexemes.items():
    for text in range(len(texts['texts'])):
        means = vm.disambiguate(lexeme, texts['norm_texts'][text])
        i = np.argmax(means)
        if lexeme not in texts_meanings.keys():
            texts_meanings[lexeme] = {}
        if i not in texts_meanings[lexeme].keys():
            texts_meanings[lexeme][i] = [texts['texts'][text]]
        else:
            texts_meanings[lexeme][i].append(texts['texts'][text])

In [15]:
lexeme_meanings = {}
for lexeme, meanings in lexemes_norm.items():
    for meaning in range(len(meanings['meanings'])):
        means = vm.disambiguate(lexeme, meanings['norm_meanings'][meaning])
        i = np.argmax(means)
        if lexeme not in lexeme_meanings.keys():
            lexeme_meanings[lexeme] = {}
        if i not in lexeme_meanings[lexeme].keys():
            lexeme_meanings[lexeme][i] = [meanings['meanings'][meaning]]
        else:
            lexeme_meanings[lexeme][i].append(meanings['meanings'][meaning])

In [16]:
right_meanings_adagram = {}

Я вывожу ненормализованные тексты, чтобы было удобнее читать, единственная проблема, получилось так, что и знаки препинания у них через пробел, но в целом это читабельно.

In [18]:
print('Значения для лексемы "среда":')
print(*list(sorted(lexeme_meanings['среда'].items())), sep = '\n')
print('\n')
print('Контексты для значения 1:')
print(*texts_meanings['среда'][1][0:5], sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['среда'][2][0:5], sep = '\n')
print('\n')
print('Контексты для значения 4:')
print(*texts_meanings['среда'][4][0:5], sep = '\n')

Значения для лексемы "среда":
(1, ['Среда - Третий (после воскресенья) день недели (середина недели).'])
(2, ['Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами. Воздушная среда. Распространение света в однородной среде. Преломляющие среды. Фильтрующие среды.', 'Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма. Географическая среда.'])
(4, ['Социально-бытовая обстановка, в которой протекает жизнь человека; окружение. [Нехлюдов] всем существом почувствовал отвращение к той своей среде, в которой он жил до сих пор. Л. Толстой, Воскресение. В судьбе, вкусах и характере каждого человека огромное значение имеет его детство, среда, в которой он вырастал, был воспитан. Соколов-Микитов, Автобиографические заметки.', 'кого или какая. Совокупность людей, связанных общностью социально-бытовых условий существования, общностью профессии, занятий. Рабочая среда. В среде учащихся. Читательская среда. □ — Из среды 

In [19]:
right_meanings_adagram['среда'] = [1, 1, 1, 1, 1,
                                   1, 1, 1, 1, 0,
                                   1, 0, 1, 1, 1]

Про **среду**: первое значение действительно вынесено отдельно в словаре, второе значение в словаре выделено как 2 разных, а третье как раз объединено через знак ||, поэтому здесь более менее нормальная разбивка по значениям.

In [21]:
print('Значения для лексемы "мотив":')
print(*list(sorted(lexeme_meanings['мотив'].items())), sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['мотив'][2][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['мотив'][3][0:5], sep = '\n')

Значения для лексемы "мотив":
(2, ['Побудительная причина, основание, повод к какому-л. действию, поступку. Кроме повара, гневались и на лакеев, прислуживавших за столом. Мотивы были самые разнообразные; не так ступил, не так подал, не так взглянул. Салтыков-Щедрин, Пошехонская старина. — Был у меня и другой мотив желать сближения с вами: сходство наших убеждений. Чернышевский, Пролог.', 'Довод в пользу чего-л. Вольф стал очень оживленно выставлять мотивы, по которым дело должно было быть кассировано. Л. Толстой, Воскресение. Сабуров еще раз повторил все мотивы, по которым он решил отложить атаку до ночи. Симонов, Дни и ночи.'])
(3, ['Составной элемент темы произведения искусства. Основной мотив повести. Вышивка по народным мотивам. □ Когда-то Пастухов купил несколько его картин — излюбленные гривнинские мотивы: вода, пепельные ветлы, дороги, уходящие в нежные дали. Федин, Костер.', 'Мотив - Муз. Простейшая ритмическая единица мелодии, состоящая обычно из двух-трех звуков, объединенных

In [22]:
right_meanings_adagram['мотив'] = [1, 1, 1, 1, 0,
                                   1, 1, 1, 1, 1]

Про **мотив**: очень хорошо разделились значения, возможно в третьем стоило выделить что-то отдельно, но в целом разделение логично.

In [24]:
print('Значения для лексемы "звезда":')
print(*list(sorted(lexeme_meanings['звезда'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['звезда'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['звезда'][2][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['звезда'][3][0:5], sep = '\n')

Значения для лексемы "звезда":
(0, ['Геометрическая фигура с остроконечными выступами, равномерно расположенными по окружности; фигура с лучами, исходящими от центра. Пятиконечная звезда. Нарисовать звезду.'])
(1, ['Звезда - Воинский значок, имеющий такую форму и носимый на фуражке, шапке и т. п.', 'Знак отличия, орден, имеющий такую форму. Маршальская звезда. □ Их нагнал строгой наружности седой господин, со звездой на фраке. Писемский, Тысяча душ. [Багратион был] в новом узком мундире с русскими и иностранными орденами и с георгиевскою звездой на левой стороне груди. Л. Толстой, Война и мир. Он резко повернулся, и при этом на лацкане его пиджака блеснула Звезда Героя Социалистического Труда. С. Антонов, Три тысячи девятнадцатая морская.'])
(2, ['О человеке, прославившемся в какой-л. сфере деятельности; о знаменитости. — Ни одной зимы не проходило без того, чтобы не приезжала какая-нибудь звезда. Чехов, Живая хронология. На него [Урманова] еще в гимназии смотрели как на будущую звезду

In [25]:
right_meanings_adagram['звезда'] = [0, 1, 0, 0, 1,
                                    0, 1, 0, 1, 1,
                                    1, 1, 1, 1, 0]

Про **звезду**: немного странно, что небесное тело не выделилось отдельно, но остально разделение более менее логично, не считая значения про пятно на лбу, но это какой-то очень узкий смысл.

In [27]:
print('Значения для лексемы "земля":')
print(*list(sorted(lexeme_meanings['земля'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['земля'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['земля'][2][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['земля'][3][0:5], sep = '\n')

Значения для лексемы "земля":
(0, ['Верхний, поверхностный слой коры нашей планеты, а также ее более глубокие слои; почва, грунт. Пахать землю. В недрах земли. □ Мертвые в землю зарыты; больные Скрыты в землянках; рабочий народ Тесной гурьбой у конторы собрался. Н. Некрасов, Железная дорога. [Виктору] нетерпеливо хотелось в шахту, или, как он теперь говорил, под землю. — Под землей будем работать! — хвастался он Андрею. Горбатов, Донбасс.', 'Поверхность, плоскость, на которой мы стоим, по которой движемся. Веселый, свежий ветер гулял над землею и в меру шумел и играл, все шевеля и ничего не тревожа. Тургенев, Первая любовь. По тебе ль, нива, ветер разгуливал, Гнул колосья твои до земли. А. К. Толстой, Уж ты нива моя, нивушка.', 'Рыхлое темно-бурое вещество, входящее в состав коры нашей планеты. Песок с землей. Комок земли. □ Бешено кидаясь на берега, оно [море] оставляет рыб, ракушки и уносит песок, землю и прочее. И. Гончаров, Фрегат «Паллада». Тут он заметил, что окоп рядом засыпан с

In [28]:
right_meanings_adagram['земля'] = [1, 1, 1, 1, 1,
                                   1, 0, 1, 0, 1,
                                   0, 1, 1, 0, 1]

Про **землю**: странно, что планета объединилась с сушей, а значения территория и страна разделились.

In [29]:
print('Значения для лексемы "светлый":')
print(*list(sorted(lexeme_meanings['светлый'].items())), sep = '\n')
print('\n')
print('Контексты для значения 1:')
print(*texts_meanings['светлый'][1][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['светлый'][3][0:5], sep = '\n')

Значения для лексемы "светлый":
(0, ['Менее насыщенный, менее густой по сравнению с основным цветом. Светлая зелень. □ Сестра была блондинка, светлая блондинка. Достоевский, Подросток.'])
(1, ['перен. Радостный, ничем не омраченный. Светлые воспоминания. Светлое будущее. □ За каждый светлый день иль сладкое мгновенье Слезами и тоской заплатишь ты судьбе. Лермонтов, Отчего. Быстро идет к нам новая, светлая жизнь. Чернышевский, Что делать? Настоящей, светлой, легкой веселости как будто никогда не было в его сердце. Достоевский, Подросток.', 'Выражающий радость, безмятежность. Светлая улыбка. □ — У вас такое веселое, светлое лицо, вы улыбаетесь. Тургенев, Дворянское гнездо.', 'Выражающий ясность духа, умиротворение. Лица торжественные, светлые, добрые; товарищи внимательны друг к другу и ласковы. Помяловский, Очерки бурсы.', 'перен. Обладающий высокими моральными качествами; возвышенный, благородный. Удивительно светел был этот человек, --- искренне веровавший в безграничную силу народа, 

In [30]:
right_meanings_adagram['светлый'] = [0, 1, 1, 1, 1,
                                     1, 0, 1, 1, 1]

Про **светлый**: некоторые переносные значения смешались с обычными, лучше бы их было выделить в отдельную категорию.

In [31]:
full_right_meanings_adagram = []
for lexeme, meanings in right_meanings_adagram.items():
    full_right_meanings_adagram.extend(meanings)
full_model_meanings_adagram = [1] * len(full_right_meanings_adagram)

In [32]:
adagram_accuracy = accuracy_score(full_right_meanings_adagram, full_model_meanings_adagram)
adagram_accuracy

0.7692307692307693

## ELMo

In [ ]:
model = ElmoModel()
model.load("196")

In [34]:
def get_elmo_vectors(word, contexts, model):
    tokens = [c for c in contexts]
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:  # если нормализация накосячила и лемму не найти
            continue
    return word_vecs

In [35]:
text_vecs = {}
for lexeme, texts in texts_with_lexemes.items():
    word_vecs = get_elmo_vectors(lexeme, texts['norm_texts'], model)
    text_vecs[lexeme] = word_vecs

In [36]:
lexeme_vecs = {}
for lexeme, meanings in lexemes_norm.items():
    word_vecs = get_elmo_vectors(lexeme, meanings['norm_meanings'], model)
    lexeme_vecs[lexeme] = word_vecs

In [68]:
kmeans_texts = {}
kmeans_meanings = {}
for lexeme, word_vecs in text_vecs.items():
    kmeans_lexeme_texts = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(word_vecs)
    kmeans_lexeme_meanings = kmeans_lexeme_texts.predict(lexeme_vecs[lexeme])
    kmeans_texts[lexeme] = kmeans_lexeme_texts
    kmeans_meanings[lexeme] = kmeans_lexeme_meanings

In [38]:
texts_meanings = {}
for lexeme, labels in kmeans_texts.items():
    texts_meanings[lexeme] = {}
    for label in range(len((labels.labels_))):
        if labels.labels_[label] not in texts_meanings[lexeme].keys():
            texts_meanings[lexeme][labels.labels_[label]] = [texts_with_lexemes[lexeme]['texts'][label]]
        else:
            texts_meanings[lexeme][labels.labels_[label]].append(texts_with_lexemes[lexeme]['texts'][label])

In [69]:
lexeme_meanings = {}
for lexeme, labels in kmeans_meanings.items():
    lexeme_meanings[lexeme] = {}
    for label in range(len(labels)):
        if labels[label] not in lexeme_meanings[lexeme].keys():
            lexeme_meanings[lexeme][labels[label]] = [lexemes[lexeme][label]]
        else:
            lexeme_meanings[lexeme][labels[label]].append(lexemes[lexeme][label])

In [40]:
right_meanings_elmo = {}

In [41]:
print('Значения для лексемы "среда":')
print(*list(sorted(lexeme_meanings['среда'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['среда'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['среда'][3][0:5], sep = '\n')

Значения для лексемы "среда":
(0, ['Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами. Воздушная среда. Распространение света в однородной среде. Преломляющие среды. Фильтрующие среды.'])
(3, ['Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма. Географическая среда.', 'Социально-бытовая обстановка, в которой протекает жизнь человека; окружение. [Нехлюдов] всем существом почувствовал отвращение к той своей среде, в которой он жил до сих пор. Л. Толстой, Воскресение. В судьбе, вкусах и характере каждого человека огромное значение имеет его детство, среда, в которой он вырастал, был воспитан. Соколов-Микитов, Автобиографические заметки.', 'кого или какая. Совокупность людей, связанных общностью социально-бытовых условий существования, общностью профессии, занятий. Рабочая среда. В среде учащихся. Читательская среда. □ — Из среды студенчества рекрутируется большинство революционеров. М. Горький, Жизнь Кл

In [43]:
right_meanings_elmo['среда'] = [0, 0, 0, 0, 0,
                                1, 1, 1, 1, 1]

Про **среду**: странно, что не отделилосб значение дня.

In [42]:
print('Значения для лексемы "мотив":')
print(*list(sorted(lexeme_meanings['мотив'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['мотив'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 1:')
print(*texts_meanings['мотив'][1][0:5], sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['мотив'][2][0:5], sep = '\n')

Значения для лексемы "мотив":
(0, ['Составной элемент темы произведения искусства. Основной мотив повести. Вышивка по народным мотивам. □ Когда-то Пастухов купил несколько его картин — излюбленные гривнинские мотивы: вода, пепельные ветлы, дороги, уходящие в нежные дали. Федин, Костер.', 'Напев, мелодия. [Райский] сел за фортепьяно и начал брать сильные аккорды ---. Перебрал мотивы из нескольких опер. И. Гончаров, Обрыв. [Музыканты] подхватывали все тот же мотив мазурки. Л. Толстой, После бала. [Зина] попыталась припомнить их [Журбиных] песню — и слова и мотив упорно от нее ускользали. Кочетов, Журбины.'])
(1, ['Побудительная причина, основание, повод к какому-л. действию, поступку. Кроме повара, гневались и на лакеев, прислуживавших за столом. Мотивы были самые разнообразные; не так ступил, не так подал, не так взглянул. Салтыков-Щедрин, Пошехонская старина. — Был у меня и другой мотив желать сближения с вами: сходство наших убеждений. Чернышевский, Пролог.'])
(2, ['Довод в пользу чег

In [44]:
right_meanings_elmo['мотив'] = [1, 1, 1, 1, 1,
                                0, 1, 0, 1, 0,
                                1, 1, 1, 1, 1]

Про **мотив**: побудительная причина и довод разделились, что странно.

In [45]:
print('Значения для лексемы "звезда":')
print(*list(sorted(lexeme_meanings['звезда'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['звезда'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['звезда'][3][0:5], sep = '\n')

Значения для лексемы "звезда":
(0, ['перен. Судьба, участь; счастье, удача. Весть о солдатчине мало тревожила его: он верил в свою звезду. Помяловский, Очерки бурсы. — Это у вас счастливая звезда. Ведь на полсантиметра правее, и была бы совсем другая картина. А вы через месяц воевать сможете. Эренбург, Буря.', 'Знак отличия, орден, имеющий такую форму. Маршальская звезда. □ Их нагнал строгой наружности седой господин, со звездой на фраке. Писемский, Тысяча душ. [Багратион был] в новом узком мундире с русскими и иностранными орденами и с георгиевскою звездой на левой стороне груди. Л. Толстой, Война и мир. Он резко повернулся, и при этом на лацкане его пиджака блеснула Звезда Героя Социалистического Труда. С. Антонов, Три тысячи девятнадцатая морская.', 'Светлое пятно на лбу животного. Конь был рослый и статный, с белой звездой на лбу. Седых, Даурия.'])
(3, ['Небесное тело, состоящее из раскаленных газов (плазмы), по своей природе сходное с Солнцем и представляющееся взору человека на н

In [46]:
right_meanings_elmo['звезда'] = [1, 1, 1, 1, 0,
                                 1, 1, 1, 1, 1]

Про **звезду**: мало групп значений, небесное тело не отделилось от остальных.

In [47]:
print('Значения для лексемы "земля":')
print(*list(sorted(lexeme_meanings['земля'].items())), sep = '\n')
print('\n')
print('Контексты для значения 0:')
print(*texts_meanings['земля'][0][0:5], sep = '\n')
print('\n')
print('Контексты для значения 1:')
print(*texts_meanings['земля'][1][0:5], sep = '\n')
print('\n')
print('Контексты для значения 2:')
print(*texts_meanings['земля'][2][0:5], sep = '\n')

Значения для лексемы "земля":
(0, ['Третья от Солнца, обитаемая нами планета (с прописной буквы). Земля движется вокруг Солнца. Окружность Земли. Луна — спутник Земли. □ От Земли до Солнца около 150 миллионов километров; пролететь это расстояние — то же, что 4000 раз объехать кругом Земли. С. Вавилов, Глаз и Солнце.', 'Место жизни и деятельности людей. Слухом земля полнится. Поговорка. [Сальери:] Все говорят: нет правды на земле, Но правды нет — и выше. Пушкин, Моцарт и Сальери. И сколько вообще обидного на земле, чего вовсе не нужно. М. Горький, В людях.', 'Суша, земная твердь (в отличие от водного или воздушного пространства). В воображении невольно возникает мысль о близости берега, земли. Гл. Успенский, Очерки переходного времени. Ночь будет страшная, и буря будет злая, Сольются в мрак и гул и небо и земля. Фет, Приметы. По времени должен был уже показаться Сахалин, но никакой земли поблизости не было видно. Чаковский, У нас уже утро.', 'Верхний, поверхностный слой коры нашей плане

In [48]:
right_meanings_elmo['земля'] = [1, 1, 0, 1, 1,
                                0, 1, 1, 0, 0,
                                1, 1, 1, 0, 1]

Про **землю**: территория и страна разделились, а планета не выделилась отдельно.

In [49]:
print('Значения для лексемы "светлый":')
print(*list(sorted(lexeme_meanings['светлый'].items())), sep = '\n')
print('\n')
print('Контексты для значения 1:')
print(*texts_meanings['светлый'][1][0:5], sep = '\n')
print('\n')
print('Контексты для значения 3:')
print(*texts_meanings['светлый'][3][0:5], sep = '\n')

Значения для лексемы "светлый":
(1, ['Хорошо освещенный, наполненный светом. Мои вещи были внесены в большую светлую комнату, выходившую большими окнами прямо на пруд. Мамин-Сибиряк, Сестры. Стояла светлая, лунная ночь, тротуары казались занесенными снегом, таким пронзительно белым был лунный свет. Павленко, Труженики мира.', 'Не темного цвета. Светлое пальто. Светлые глаза. Светлые обои. □ Вошел полицейский чиновник красивой наружности, с бакенбардами не слишком светлыми и не темными. Гоголь, Нос. Здешняя осина имеет столь светлую кору, что я сначала принимал ее за березу. Арсеньев, В горах Сихотэ-Алиня.', 'Менее насыщенный, менее густой по сравнению с основным цветом. Светлая зелень. □ Сестра была блондинка, светлая блондинка. Достоевский, Подросток.', 'Чистый, прозрачный (о жидкости). Днепра ж светлы стремнины, Чиста его вода, Не видано в нем тины От веку никогда! А. К. Толстой, Сватовство. Лиза проворно поднялась и отерла глаза, на которых сияли светлые, непролившиеся слезы. Турген

In [50]:
right_meanings_elmo['светлый'] = [1, 1, 1, 1, 1,
                                  1, 1, 1, 1, 1]

Про **светлый**: плохо разделились значения, у светлого должно быть больше различия (поэтому везде единицы получились, слишком много смешения значений).

In [51]:
full_right_meanings_elmo = []
for lexeme, meanings in right_meanings_elmo.items():
    full_right_meanings_elmo.extend(meanings)
full_model_meanings_elmo = [1] * len(full_right_meanings_elmo)

In [52]:
elmo_accuracy = accuracy_score(full_right_meanings_elmo, full_model_meanings_elmo)
elmo_accuracy

0.7666666666666667

Результаты в целом получились одинаковые, но можно заметить, что ELMo делает меньше различий в значениях, часто в одну цифру попадают очень разные значения, из-за чего технически получается 1 в разметке, но логически понятно, что разделение значений здесь хуже. По идее ELMo должна работать лучше, но возможно здесь получаются такие результаты, потому что разделять так словарные значения достаточно сложно. Чтобы использовать одни и те же индексы мне пришлось использовать predict (возможно есть другой способ, но я не смогла придумать), и возможно это повлияло на работу ELMo. Если делать не через predict, а так же, как и сами тексты, то значений там действительно получается больше и скорее всего даже лучше разделено, но проблема в том, что индексы значений и индексы текстов будут отличаться, они не будут связаны друг с другом из-за кластеризации. Еще, кстати, одно замечание, сложно подобрать, на какое количество кластеров нужно делить, в идеале нужно для каждого слова это подбирать отдельно, но здесь я вроде бы нашла средний вариант, но все равно это стоит отметить. (Делать количество кластеров равным количеству значений не вариант, потому что хотелось бы все-таки некоторые значения объединить, особенно те, которые в словаре написаны через ||.) AdaGram же кластеризирует сразу, поэтому возможно здесь это сработало лучше.


Хотелось бы также отметить, что я пробовала работу моделей на нескольких других словах и иногда в ELMo вообще получалось так, что все значения попадали в один кластер, что очень плохо для сравнения моделей, поэтому я убирала такие слова. Возможно можно улучшить метод кластеризации.

Сложность с примерами:

*   В **звездах** встречается вот такой пример: *Начальников с большими звёздами , какие в плен попадали , выкупили за деньги , за оружие выменяли , чтоб этим оружием наших убивать .* И как я поняла, модели часто относят это к значению звезда в смысле человек (возможно из-за слова начальник). Но здесь скорее имеется в виду награда или воинский значок, и в ELMo как раз получилась 1, потому что много значений попало в один кластер (хотя ELMo отнес эти два значения в разные кластеры), в AdaGram у этого примера 0.
*   В **светлом** есть такой пример: *А белые чайки , как светлые души , будут виться над кипящим морем .* Он относится к значению светлый в смысле дух, душа, но AdaGram отнес его к другому значению, а ELMo смешал много значений у этого слова, поэтому пришлось поставить 1, но вообще со значениями этого слова в ELMo не очень хорошо получилось.
